## 1. Split the train dataset file

In [ ]:
import pandas as pd

file_path = 'train.xlsx'
df = pd.read_excel(file_path)
split_n = 10

rows_per_file = len(df) // split_n  
remaining_rows = len(df) % split_n

start_row = 0

for i in range(split_n):
    end_row = start_row + rows_per_file
    if remaining_rows > 0:
        end_row += 1
        remaining_rows -= 1
    split_df = df.iloc[start_row:end_row, [0,2,4,5]]
    split_df.to_excel(f'train_{i+1}.xlsx', index=False)
    start_row = end_row

## 2. Translate the document from Korean to English using "[Google Translate]("https://translate.google.co.kr/?sl=auto&tl=ko&op=translate")"

## 3. Merge the files and save in JSON format

In [40]:
import pandas as pd
import json

# 엑셀 파일 읽기
file_path = 'train.xlsx'
train_df = pd.read_excel(file_path)
split_n = 10

# 병합할 파일 목록
file_list = [f'train_en_{i+1}.xlsx' for i in range(split_n)]

# 빈 데이터프레임 생성
merged_df = pd.DataFrame()

# 각 파일을 읽어서 병합
for file in file_list:
    df_en = pd.read_excel(file)
    merged_df = pd.concat([merged_df, df_en], ignore_index=True)

train_columns = ['image_id', 'image']

train_df = train_df[train_columns]
final_df = pd.merge(merged_df, train_df, on='image_id', how='left')
final_df = final_df.drop_duplicates()
print("Columns in final_df:", final_df.columns)

json_list = []
for _, row in final_df.iterrows():
    json_entry = {
        "image": row['image'],
        "question": row[' question'].strip(),
        "answers": [
            {
                "answer": str(row[' answer']).strip(),
                "answer_confidence": "yes"  
            },
        ],
        "answer_type": row[' category'].strip(),  # 예제 데이터에 따라 변경 가능
        "answerable": 1  # 예제 데이터에 따라 변경 가능
    }
    json_list.append(json_entry)

# JSON 형식으로 저장
with open('train_en.json', 'w') as json_file:
    json.dump(json_list, json_file, indent=2)

Columns in final_df: Index(['image_id', ' category', ' question', ' answer', 'image'], dtype='object')
